
#### Nama: Aurellia Vieryn
#### NIM : 2440066626

### UTS - TEXT MINING

Untuk ujian pada kali ini, dibutuhkan dataset yang berisi teks berita dari berbagai portal. Teks ini kemudian akan diolah menjadi vektor setelah dilakukan tahap preprocessing. Selain itu, akan dicek juga 5 nilai TF-IDF tertinggi untuk setiap datanya. <br> <br>

Dataset terdiri dari 3 kolom, yaitu: Teks, Media, dan Label. `Teks` merupakan isi dari beritanya, `Media` adalah portal beritanya, dan `Label` adalah jenis beritanya (Politik, Hiburan, atau Olahraga). Datasetnya memiliki 12 data, dengan berbagai jenis label, serta dari beberapa portal berita (Kompas, Detik, Liputan 6, dan Tempo).<br><br> 

Pertama-tama, dataset yang telah dibuat akan diunggah ke file Colabnya terlebih dahulu.

In [1]:
from google.colab import files
 
uploaded = files.upload()

Saving dataset.xlsx to dataset (1).xlsx


Karena datasetnya dibuat pada Excel, maka akan digunakan library pandas untuk membaca filenya.

In [2]:
import pandas as pd

df = pd.read_excel('dataset.xlsx')

### Preprocessing

Pada tahap ini, ada beberapa task yang harus dilakukan kepada teksnya untuk penilaian, yaitu:
- Pembersihan Link (https, .com, dll)
- Pembersihan Hashtag
- Pembersihan Simbol
- Pembersihan Punctuations
- Ubah menjadi Lowercase
- Penghilangan Stopwords
- Vectorization
- TF-IDF

<br>

Sebelum memulai, file yang sudah dijadikan dataframe akan dicek terlebih dahulu hanya untuk memastikan.

In [3]:
df.head()

,Teks,Media,Label
0,Semarang - Menparekraf Sandiaga Uno mengucapka...,Detik,Politik
1,Inter Milan menjamu Juventus pada leg kedua se...,Liputan 6,Olahraga
2,The Flash diadaptasi dari karakter DC Comics d...,Tempo,Hiburan
3,Pemusatan latihan nasional (pelatnas) tahap ke...,Tempo,Olahraga
4,"Wali Kota Depok, Mohammad Idris mengakui bahwa...",Kompas,Politik


#### Remove Link (https and .com)

Untuk penghapusan link pada teks, dapat digunakan metode dari library `re `(regular expression) yaitu metode replace atau subsitusi. Functionnya adalah `re.sub()`, dimana akan dicari urutan teks yang berbentuk link lalu diganti oleh kosong. <br><br>

Pada soal, dikatakan untuk menghilangkan seluruh kata yang berawalan dengan https atau berakhiran dengan .com. Maka dari itu, akan dimulai dengan penghapusan kata-kata yang berawalan dengan https. Untuk https sendiri, kita akan mencari kata tersebut dengan menggunakan code `https?:\/\/(\S*)(\s+)` dalam parameter fungsinya. <br><br>

Secara rinci, codenya bekerja sebagai berikut:
- `?` setelah `https` : untuk kata-kata yang berawalan https, atau kata-kata yang bisa dibentuk dari kombinasi abjad h, t, t, p, s
- `:` : untuk menandakan harus terdapat `:` dalam teksnya
- `\/\/` : untuk menandakan harus terdapat `//` dalam teksnya
- `(\S*)` : untuk set karakter yang tidak memiliki minimal karakter dan bukan whitespace
- `(\s+)` : untuk set whitespace yang memiliki minimal satu karakter <br>

Jadi, teks yang dihapus adalah teks dengan awalan `https` dan kombinasinya, lalu harus dilanjutkan dengan `:` dan `//` dan berakhiran apapun kecuali whitespace sebelum titik. Harus dihapus setelah terdeteksi adanya minimal satu whitespace. `re.sub()` juga akan melewati loop karena terdapat lebih dari 1 row pada dataset.

In [4]:
import re
# https
df.Teks = [re.sub(r'https?:\/\/(\S*)(\s+)', '', i) for i in df.Teks]

Untuk yang berakhiran dengan `.com`, akan mengalami penambahan. Penghapusan pada semua kata yang memiliki `.` akan dilakukan karena pastinya masih terdapat link-link yang lain yang bukan berakhiran dengan `.com`. Tentunya hal ini tidak 100% akurat, namun lebih baik untuk dihapus karena kebanyakan merupakan link. <br><br>

Secara rinci, code `(\S+)[.](\S+)(\s+)` bekerja sebagai berikut:
-  `(\S+)` : untuk set karakter yang bukan whitespace yang memiliki minimal satu karakter 
- `[.]` : untuk menandakan harus terdapat `.` dalam teksnya
- `(\s+)` : untuk set whitespace yang memiliki minimal satu karakter

Jadi, teks yang dihapus adalah teks dengan awalan apapun kecuali whitespace sebelum titik, dan juga berakhiran apapun kecuali whitespace sebelum titik. Harus dihapus setelah terdeteksi adanya minimal satu whitespace. `re.sub()` juga akan melewati loop karena terdapat lebih dari 1 row pada dataset.

In [5]:
# Any Word that has . in it
df.Teks = [re.sub(r'(\S+)[.](\S+)(\s+)', '', i) for i in df.Teks]

#### Remove Hashtag

Untuk penghapusan hashtag pada teks, dapat juga digunakan metode regex dengan library `re`. Sama dengan sebelumnya, menggunakan `re.sub()` dan looping untuk tiap row.<br><br>

Secara rinci, code `#(\S+)(\s+)` bekerja sebagai berikut:
- `#`: untuk kata yang berawalan dengan `#`
-  `(\S+)` : untuk set karakter yang bukan whitespace yang memiliki minimal satu karakter 
- `(\s+)` : untuk set whitespace yang memiliki minimal satu karakter

Jadi, teks yang dihapus adalah teks dengan awalan `#` dan berakhiran apapun kecuali whitespace sebelum titik. Harus dihapus setelah terdeteksi adanya minimal satu whitespace. 

In [6]:
# Hashtag
df.Teks = [re.sub(r'#(\S+)(\s+)', '', i) for i in df.Teks]

### Remove Punctuations and Symbols (Non ASCII)

Untuk penghapusan seluruh punctuations (tanda baca), dapat digunakan fungsi `.maketrans()` yang terdapat pada library `string`. Dibutuhkan juga fungsi `translate()` dimana fungsi ini akan mnegembalikan sebuah string yang beberapa karakternya akan diganti oleh karakter pada mapping table (`maketrans()`). Mapping table ini berisi untuk tentang penggantian setiap tanda baca yang terdaftar pada `string.punctuations` menjadi kosong. Tidak lupa juga untuk menjalankan code ini dalam looping.

In [7]:
import string
# Punctuations
df.Teks = [word.translate(str.maketrans('', '', string.punctuation)) for word in df.Teks]

Karena pada teks berita terkadang terdapat quotation marks ("), maka metode ini harus dijalankan. Bisa juga terdapat simbol-simbol. Seluruh hal tersebut termasuk ke golong karakter yang bukan ASCII. Masalah ini dapat diselesaikan dengan membuat sebuah function manual, dimana tiap karakter pada string akan diseleksi. Jika karakter tersebut memiliki ASCII kurang dari 128, maka tidak ada masalah. Namun, jika karakternya memiliki ASCII lebih dari 128, maka akan dikembalikan kosong. <br><br>

Pre-defined function tersebut akan dipanggil untuk setiap row dalam sebuah looping.

In [8]:
# Symbols or Non-ASCII Characters
def remove_non_ascii(string):
    return ''.join(
      char for char in string if ord(char) < 128
    )

for i in df.index:
  df["Teks"][i]= remove_non_ascii(df["Teks"][i])

#### Remove Numbers

Penghapusan angka juga bisa dilakukan dengan fungsi yang sama dengan penghapusan link dan hashtag. Kali ini yang akan digunakan adalah `\d+` yaitu kombinasi angka dengan angkanya minimal 1 dan akan diganti dengan kosong.

In [9]:
df.Teks = [re.sub(r"\d+","", i) for i in df.Teks]

#### Remove Multiple Whitecases

Sama juga, akan digunakan fungsi yang sama untuk penghapusan spasi yang berulang. Di sini yang akan diganti adalah whitespaces minimal 1 dan akan digantikan dengan sebuah whitespace.

In [10]:
df.Teks = [re.sub(r"\s+"," ", i) for i in df.Teks]

#### Make Text to Lowercase

Langkah terakhir sebelum masuk ke proses tokenisasi dan vektorisasi adalah pengubahan teks menjadi lowercase (huruf kecil). Fungsi yang dapat digunakan terdapat pada library string, yaitu `str.lower()`

In [11]:
df.Teks = df.Teks.str.lower()

Hasil gabungannya akan kira-kira menjadi seperti ini:

In [12]:
print("\n".join(df["Teks"]))

semarang menparekraf sandiaga uno mengucapkan selamat kepada gubernur jawa tengah ganjar pranowo secara langsung saat ditanya soal pilpres mereka bertemu di kota semarang dalam acara terkait ekonomi desa ganjar dan sandiaga hari ini menjadi pembicara dalam rapat koordinasi daerah dan pleno tpkad sejawa tengah dengan tema percepatan pemulihan dan perkembangan ekonomi jawa tengah melalui kemandirian ekonomi desa di hotel gumaya semarang usai acara digelar jumpa pers soal acara tersebut kemudian keduanya ditanya soal politik termasuk sandiaga yang santer dikabarkan akan merapat ke ppp sedangkan ppp baru saja mendeklarasikan mengusung ganjar sebagai capres saat ditanya soal dukungan itu sandiaga langsung mengucapkan selamat selamat lho selamat kata sandi sambil melihat ganjar yang berjalan di sampingnya saat keluar dari hotel gumaya kamis menanggapi itu ganjar melempar pertanyaan sembari bercanda ia menanyakan apakah sandiaga sudah memiliki kartu tanda anggota kta dalam hal ini kta ppp ema

#### Tokenization

Tokenisasi adalah proses pemecahan sebuah string menjadi berbagai sub-string. Proses ini bisa dilakukan menggunakan library nltk dengan package punkt dengan fungsi `word_tokenize`. Package punkt harus diunduh terlebih dahulu.

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

 Yang akan diapply dengan functionnya adalah setiap row Teks, seperti yang dapat dilihat pada code yang tertera di bawah.

In [14]:
from nltk.tokenize import word_tokenize

df.Teks = df.apply(lambda row: nltk.word_tokenize(row.Teks), axis=1)

Hasil dari yang telah dipisah-pisah pada dataframe dapat dilihat:

In [15]:
df.head()

,Teks,Media,Label
0,"[semarang, menparekraf, sandiaga, uno, menguca...",Detik,Politik
1,"[inter, milan, menjamu, juventus, pada, leg, k...",Liputan 6,Olahraga
2,"[the, flash, diadaptasi, dari, karakter, dc, c...",Tempo,Hiburan
3,"[pemusatan, latihan, nasional, pelatnas, tahap...",Tempo,Olahraga
4,"[wali, kota, depok, mohammad, idris, mengakui,...",Kompas,Politik


#### Remove Stopwords

Stopwords adalah kata-kata yang tidak memiliki makna yang signifikan, dan tidak penting untuk dianalisis. Maka dari itu, stopwords ini harus dibuang untuk kepentingan ke depannya. Dapat digunakan fungsi dari library nltk juga dengan package yang perlu diunduh terlebih dahulu, yaitu package `stopwords`.

In [16]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Selain bahasa Indonesia, terkadang teks berita di Indonesia juga sering menggunakan bahasa Inggris. Dengan demikian, akan dihapus stopwords-stopwords pada bahasa Indonesia dan bahasa Inggris. List dari stopwords ini dapat diakses melalui function `stopwords.words()` dengan bahasa yang sesuai pada fungsinya.

In [17]:
list_stopwords = set(stopwords.words('indonesian'))
list_stopwords2 = set(stopwords.words('english'))

Setelah sudah dilist semua stopwordsnya, maka dapat digunakan. Caranya adalah dengan mengiterasi setiap kata, dan jika kata tersebut tidak terdapat pada list stopwords, maka akan ditambahkan.

In [18]:
df.Teks = df.Teks.apply(lambda x: [item for item in x if item not in list_stopwords])
df.Teks = df.Teks.apply(lambda x: [item for item in x if item not in list_stopwords2])

Hasilnya dapat dilihat seperti di bawah, dimana kata-kata seperti `pada`, `the`, sudah terhapus.

In [19]:
df.head()

,Teks,Media,Label
0,"[semarang, menparekraf, sandiaga, uno, selamat...",Detik,Politik
1,"[inter, milan, menjamu, juventus, leg, semifin...",Liputan 6,Olahraga
2,"[flash, diadaptasi, karakter, dc, comics, nama...",Tempo,Hiburan
3,"[pemusatan, latihan, nasional, pelatnas, tahap...",Tempo,Olahraga
4,"[wali, kota, depok, mohammad, idris, mengakui,...",Kompas,Politik


#### Stemming

Stemming adalah tahap pengubahan kata menjadi kata dasar. Misalnya, `pembuatan` menjadi `buat`. Metode ini baik untuk digunakan karena dapat mengklasifikasi berbagai model kata menjadi satu kata. Namun, kekurangannya adalah metode stemming tidak memperhatikan struktur bahasa dan juga kadang tidak jelas hasilnya. <br> <br>

Untuk stemming bahasa Indonesia, dapat menggunakan library Sastrawi. Library ini harus diinstall terlebih dahulu.

In [20]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Fungsi yang akan dipakai adalah `StemmerFactory()` dan `create_stemmer()` untuk pembuatan stemmer-stemmer yang terdapat pada librarynya. 

In [21]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

Setelah itu, dapat mengubah seluruh kata satu per satu jika kata tersebut dapat dipotong dengan fungsi `stem()`. Contoh dari hasilnya adalah `pelatihan` yang dipotong menjadi `latih`.

In [22]:
df.Teks = df.Teks.apply(lambda x: [stemmer.stem(y) for y in x])
df.head()

,Teks,Media,Label
0,"[semarang, menparekraf, sandiaga, uno, selamat...",Detik,Politik
1,"[inter, mil, jamu, juventus, leg, semifinal, c...",Liputan 6,Olahraga
2,"[flash, adaptasi, karakter, dc, comics, nama, ...",Tempo,Hiburan
3,"[pusat, latih, nasional, pelatnas, tahap, caba...",Tempo,Olahraga
4,"[wali, kota, depok, mohammad, idris, aku, wila...",Kompas,Politik


### Text Representation

Text Representation adalah metode yang melakukan pengubahan teks menjadi angka sehingga dapat dikomputasi secara matematis. Di tahap ini, teks akan diubah menjadi vektor dengan metode `Word2Vec` dari `gensim`. Model Word2Vec yang digunakan adalah yang berjenis `skipgram` karena datasetnya tidak begitu besar sehingga dapat lebih merepresentasikan kata-kata yang jarang. Skipgram juga merupakan kata-kata yang memprediksi konteks.


In [23]:
import gensim
from gensim.models import Word2Vec 

#### Vectorization
Pada soal, diminta untuk membuat vektor dengan ukuran 50 dan frekuensi kata yang diperhitungkan minimal 2. Kedua ini masuk ke parameter `Word2Vec()` pada `vector_size` dan `min_count`. Untuk `window` (kata-kata terdekat) diatur menjadi 5 kata, dan `sg = 1` karena model yang digunakan adalah skipgram. Data yang digunakan tentunya adalah teksnya.

In [24]:
model_skipgram = gensim.models.Word2Vec(df.Teks, min_count=2 , vector_size=50, window=5, sg=1) 

Kemudian, dengan model tersebut juga kita dapat membuat list vocab dari hasil training modelnya dan diberi sebuah index pada tiap vocabnya. Fungsinya untuk memberi indexnya adalah `wv.key_to_index`.

In [25]:
vocabulary_skipgram = model_skipgram.wv.key_to_index
print(vocabulary_skipgram)

{'tim': 0, 'ganjar': 1, 'kota': 2, 'main': 3, 'dukung': 4, 'depok': 5, 'kpk': 6, 'pmwi': 7, 'ppp': 8, 'hidup': 9, 'terima': 10, 'menang': 11, 'acara': 12, 'politik': 13, 'pdip': 14, 'korupsi': 15, 'idris': 16, 'mobile': 17, 'pubg': 18, 'kamis': 19, 'masyarakat': 20, 'mil': 21, 'jessica': 22, 'partai': 23, 'hasto': 24, 'tahap': 25, 'marquez': 26, 'kardashian': 27, 'barry': 28, 'jenderal': 29, 'sandiaga': 30, 'juventus': 31, 'hasil': 32, 'jalan': 33, 'ali': 34, 'inter': 35, 'romantic': 36, 'selamat': 37, 'pranowo': 38, 'rawan': 39, 'dr': 40, 'kim': 41, 'bentuk': 42, 'motogp': 43, 'woo': 44, 'jin': 45, 'dunia': 46, 'miskin': 47, 'zod': 48, 'iskandar': 49, 'temu': 50, 'sea': 51, 'saing': 52, 'pikir': 53, 'rumah': 54, 'kali': 55, 'games': 56, 'sakit': 57, 'juli': 58, 'juara': 59, 'kembang': 60, 'allstars': 61, 'capres': 62, 'cegah': 63, 'april': 64, 'resmi': 65, 'ekonomi': 66, 'ubah': 67, 'karakter': 68, 'drama': 69, 'flash': 70, 'doldam': 71, 'kasih': 72, 'pilih': 73, 'pusat': 74, 'ulang':

Selain itu, juga dapat dilihat kemiripan antar 2 kata menggunakan fungsi `wv.similarity` dan akan keluar sebuah angka yang menandakan korelasinya. Semakin dekat dengan angka 1 maka kedua kata tersebut lebih mirip.

In [26]:
print(model_skipgram.wv.similarity('dukung','politik'))

0.76353437


In [27]:
print(model_skipgram.wv.similarity('menang','sutradara'))

0.16459744


Kita juga dapat melihat kata-kata yang paling mirip dengan suatu kata, dan ini bisa menjadi penting untuk analisis. Fungsinya adalah `wv.most_similar`.

In [28]:
similar_words = model_skipgram.wv.most_similar("ganjar")
print(similar_words)

[('tim', 0.7325751185417175), ('dukung', 0.7272816896438599), ('tahap', 0.7268189787864685), ('mana', 0.7164531946182251), ('hasil', 0.7050018906593323), ('sandiaga', 0.702980637550354), ('ucap', 0.7010056376457214), ('allstars', 0.6977913975715637), ('laku', 0.6942546963691711), ('partai', 0.689540684223175)]


Vektor bisa didapatkan dari tiap kata di model dengan fungsi `get_vector` dan kata tersebut ada di list vocabulary modelnya. Vektor ini nantinya akan menjadi sebuah 2D array.

In [29]:
vector_skipgram = [model_skipgram.wv.get_vector(word) for word in vocabulary_skipgram]

Contoh hasil dari vektornya adalah:

In [30]:
vector_skipgram[1]

array([-0.0089997 ,  0.01018937, -0.00717107, -0.00204993,  0.0204462 ,
       -0.02197101,  0.01938751,  0.00981805, -0.02933544,  0.02548002,
       -0.00349201, -0.01676376, -0.00621174, -0.00458093, -0.01285722,
        0.02169628,  0.00465443,  0.01495319, -0.02733318, -0.00076629,
        0.00674966,  0.02716178,  0.00915027, -0.03090525,  0.02230725,
        0.0073541 ,  0.00992986, -0.0113108 , -0.02865857, -0.01310587,
        0.00854186,  0.01059542,  0.00098696,  0.01874532, -0.02368223,
        0.0192348 ,  0.02101002, -0.00916815,  0.00782488, -0.00511625,
        0.01977342, -0.00662259,  0.00471749,  0.01191002,  0.04684968,
        0.01669906, -0.02339885, -0.03178798,  0.01348775,  0.00306686],
      dtype=float32)

#### TF-IDF 

TF-IDF (Term Frequency - Inverse Document Frequency) adalah metode yang bisa memprediksi seberapa relevan sebuah kata untuk dokumennya. Di sini, akan dilakukan metode TFIDF pada setiap teks berita dan akan dicari 5 kata yang paling relevan bagi berita tersebut.<br><br>

Fungsi yang bisa digunakan adalah `TfidVectorizer()` yang mengubah data menjadi matrix yang berisi fitur-fitur TF-IDF.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vectorizer = TfidfVectorizer()

Untuk mencari top 5 dengan nilai TF-IDF tertinggi, pertama dibuat sebuah list yang bisa menampung dalam looping. Kita set n menjadi 5 karena ingin mencari 5 teratas.


In [33]:
import numpy as np

In [34]:
top_n = []
n = 5

Kemudian, pada loopnya, pertama datanya harus difit dan ditransform terlebih dahulu. Setelah itu, akan digunakan fungsi `get_feature_names_out()` agar dapat didapatkan nama dari fitur-fitur yang terseleksi (kata-kata). Kita juga harus menentukan urutan sortnya, lalu diambil 5 fitur (kata) yang teratas. 

In [35]:
for i in range(0,11):
  X = vectorizer.fit_transform(df['Teks'][i])
  # print(X)
  feature_array = np.array(vectorizer.get_feature_names_out())
  # print(feature_array)
  tfidf_sorting = np.argsort(X[i].toarray()).flatten()[::-1]
  # print(tfidf_sorting)
  top_n.append(feature_array[tfidf_sorting][:n])

Semua list yang telah diappend akan digabung menjadi sebuah 2D array.

In [36]:
top_all = np.array(top_n)

Berikut adalah hasil dari 5 hasil TF-IDF terbesar dari setiap data. Baris pertama merupakan data pertama, dan selanjutnya.

In [37]:
print(top_all)

[['semarang' 'usung' 'kait' 'imbuh' 'insyaallah']
 ['mil' 'wib' 'hilang' 'juara' 'jamu']
 ['karakter' 'zod' 'hodson' 'comics' 'dc']
 ['pelatnas' 'west' 'harap' 'isi' 'indonesia']
 ['idris' 'wilayah' 'jadi' 'ide' 'imbuh']
 ['repsol' 'wsbk' 'marc' 'hrc' 'iker']
 ['dr' 'jenius' 'drama' 'dunia' 'gabung']
 ['ikut' 'video' 'uvalde' 'goda' 'grasi']
 ['elite' 'lebur' 'gabung' 'ganjar' 'gubernur']
 ['iskandar' 'ya' 'csb' 'lapor' 'korban']
 ['pmwi' 'world' 'hasil' 'hitung' 'ikut']]
